<a href="https://colab.research.google.com/github/zhangxs131/New_word_discovery/blob/main/new_word_discovery_with_information_entropy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#    基于左右信息熵的新词发现算法

源代码来自苏神的科学空间 https://spaces.ac.cn/archives/3491

目前无聊的时候偶尔从前往后看看苏神博客，有意思的代码就跑一下

数据集，苏神使用的是天龙八部，这里我打算使用斗罗大陆，个人比较喜欢但估计会比天龙八部跑的慢些。

In [9]:
import numpy as np
import pandas as pd
import re
from numpy import log,min

f = open('0009009.斗罗大陆.txt', 'r',encoding='gbk') #读取文章
s = f.read() #读取为一个字符串

#定义要去掉的标点字
drop_dict = [u'，', u'\n', u'。', u'、', u'：', u'(', u')', u'[', u']', u'.', u',', u' ', u'\u3000', u'”', u'“', u'？', u'?', u'！', u'‘', u'’', u'…']
for i in drop_dict: #去掉标点字
    s = s.replace(i, '')

#为了方便调用，自定义了一个正则表达式的词典
myre = {2:'(..)', 3:'(...)', 4:'(....)', 5:'(.....)', 6:'(......)', 7:'(.......)'}

min_count = 10 #录取词语最小出现次数
min_support = 30 #录取词语最低支持度，1代表着随机组合
min_s = 4 #录取词语最低信息熵，越大说明越有可能独立成词
max_sep = 4 #候选词语的最大字数
t=[] #保存结果用。

t.append(pd.Series(list(s)).value_counts()) #逐字统计
tsum = t[0].sum() #统计总字数
rt = [] #保存结果用

for m in range(2, max_sep+1):
    print(u'正在生成%s字词...'%m)
    t.append([])
    for i in range(m): #生成所有可能的m字词
        t[m-1] = t[m-1] + re.findall(myre[m], s[i:])
    
    t[m-1] = pd.Series(t[m-1]).value_counts() #逐词统计
    t[m-1] = t[m-1][t[m-1] > min_count] #最小次数筛选
    tt = t[m-1][:]
    for k in range(m-1):
        qq = np.array(list(map(lambda ms: tsum*t[m-1][ms]/t[m-2-k][ms[:m-1-k]]/t[k][ms[m-1-k:]], tt.index))) > min_support #最小支持度筛选。
        tt = tt[qq]
    rt.append(tt.index)

def cal_S(sl): #信息熵计算函数
    return -((sl/sl.sum()).apply(log)*sl/sl.sum()).sum()

for i in range(2, max_sep+1):
    print(u'正在进行%s字词的最大熵筛选(%s)...'%(i, len(rt[i-2])))
    pp = [] #保存所有的左右邻结果
    for j in range(i+2):
        pp = pp + re.findall('(.)%s(.)'%myre[i], s[j:])
    pp = pd.DataFrame(pp).set_index(1).sort_index() #先排序，这个很重要，可以加快检索速度
    index = np.sort(np.intersect1d(rt[i-2], pp.index)) #作交集
    #下面两句分别是左邻和右邻信息熵筛选
    index = index[np.array(list(map(lambda s: cal_S(pd.Series(pp[0][s]).value_counts()), index))) > min_s]
    rt[i-2] = index[np.array(list(map(lambda s: cal_S(pd.Series(pp[2][s]).value_counts()), index))) > min_s]

#下面都是输出前处理
for i in range(len(rt)):
    t[i+1] = t[i+1][rt[i]]
    t[i+1].sort_values(ascending = False)

#保存结果并输出
pd.DataFrame(pd.concat(t[1:])).to_csv('result.txt', header = False)

正在生成2字词...
正在生成3字词...
正在生成4字词...
正在进行2字词的最大熵筛选(5360)...
正在进行3字词的最大熵筛选(6469)...
正在进行4字词的最大熵筛选(5898)...


In [14]:
with open('result.txt','r') as f:
  content=f.readlines()

words=[]
score=[]
for i in content:
  words.append(i.split(',')[0])
  score.append(int(i.split(',')[1]))

df=pd.DataFrame({'words':words,'score':score})
df=df.sort_values(by="score",ascending=False)
for i in df['words']:
  print(i)

唐三
自己
已经
没有
他们
小舞
我们
虽然
现在
此时
大师
能够
同时
戴沐白
奥斯卡
有些
可以
起来
瞬间
宁荣荣
完全
一般
马红俊
竟然
开始
武魂殿
突然
弗兰德
修炼
顿时
并没有
似乎
比比东
面前
唐昊
而且
拥有
对于
千仞雪
这种
众人
老师
直接
背后
面对
使用
朱竹清
如此
泰坦
胖子
看上去
立刻
独孤博
带着
朝着
骤然
赵无极
再次
全部
所以
通过
想要
八蛛矛
终于
更加
胡列娜
宗门
哪怕是
原本
进行
极为
波赛西
作为
渐渐
除了
对方
宁风致
昊天宗
明显
进入
柳二龙
仿佛
先前
今天
在空中
缓缓
以及
重新
准备
武魂帝国
就像是
大家
白沉香
曾经
双手
如同
它们
尤其是
赶忙
相比
火舞
继续
十分
宛如
杨无敌
凭借着
再加上
刚刚
也无法
悄然
千雪
风笑天
一旦
伴随着
牛皋
雪清河
雪崩
至于
按照
乃是
泰隆
白鹤
雪夜大帝
泰坦巨猿
唐啸
彻底
并且
可以说是
海马斗罗
楼高
玉天恒
吉祥
暂时
邪月
即将
七长老
包括
也不可能
海龙斗罗
由于
甚至连
各自
泰诺
秦明
阿银
金鳄斗罗
孟依然
迅速
别说是
王圣
负责
思龙


可以看出，大部分的人名都识别出来了，可以通过调整信息熵参数进行改变。